In [18]:
import cv2
import numpy as np
import mss
import time
import pyautogui
from ultralytics import YOLO

UP_KEY='w'
DOWN_KEY='s'
LEFT_KEY='a'
RIGHT_KEY='d'
DEADZONE=30
HOLD_TIME=0.06
CLICK_DIST=40  # Distance threshold for clicking
CLICK_COOLDOWN=1.0  # Cooldown between clicks in seconds

monitor={"top":0,"left":1100,"width":840,"height":1200}
model=YOLO("../models/v4/best.pt")
pyautogui.PAUSE=0.01
pyautogui.FAILSAFE=True
sct=mss.mss()
auto_enabled=True
last_click_time=0.0

def move_keys(keys,hold=HOLD_TIME):
    if not keys:
        return
    try:
        for k in keys:
            pyautogui.keyDown(k)
        time.sleep(hold)
    finally:
        for k in keys:
            try:
                pyautogui.keyUp(k)
            except Exception:
                pass

def click_on_jasmine(jasmine_center):
    """Click on jasmine object with proper screen coordinates"""
    global last_click_time

    # Convert local coordinates to screen coordinates
    screen_x = monitor["left"] + int(jasmine_center[0])
    screen_y = monitor["top"] + int(jasmine_center[1])

    try:
        print(f"Clicking jasmine at screen coordinates: ({screen_x}, {screen_y})")
        pyautogui.click(screen_x, screen_y, button='left')
        last_click_time = time.time()
        return True
    except Exception as e:
        print(f"Click failed: {e}")
        return False

try:
    print("Starting auto-pilot. Press 'q' to quit, 'p' to toggle auto mode")
    print(f"Current settings: CLICK_DIST={CLICK_DIST}, CLICK_COOLDOWN={CLICK_COOLDOWN}")

    while True:
        frame=np.array(sct.grab(monitor))
        frame=cv2.cvtColor(frame,cv2.COLOR_BGRA2BGR)
        results=model(frame,conf=0.7,verbose=False)

        himself_center=None
        jasmine_center=None

        # Detect objects
        for r in results:
            for box in r.boxes:
                cls=int(box.cls[0])
                label=model.names[cls]
                x1,y1,x2,y2=box.xyxy[0].cpu().numpy()
                cx,cy=(x1+x2)/2,(y1+y2)/2

                if label=="himself":
                    himself_center=(cx,cy)
                    cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(200,200,0),2)
                    cv2.putText(frame,"himself",(int(x1),int(y1)-6),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,200,0),2)
                elif label=="jasmine":
                    jasmine_center=(cx,cy)
                    cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,200,200),2)
                    cv2.putText(frame,"jasmine",(int(x1),int(y1)-6),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,200,200),2)

        # Process when both objects are detected
        if himself_center and jasmine_center:
            dx=jasmine_center[0]-himself_center[0]
            dy=jasmine_center[1]-himself_center[1]
            dist=(dx**2+dy**2)**0.5

            # Visual feedback
            cv2.circle(frame,(int(himself_center[0]),int(himself_center[1])),5,(200,200,0),-1)
            cv2.circle(frame,(int(jasmine_center[0]),int(jasmine_center[1])),5,(0,200,200),-1)
            cv2.line(frame,(int(himself_center[0]),int(himself_center[1])),(int(jasmine_center[0]),int(jasmine_center[1])),(0,255,0),1)

            # Show distance and click zone
            cv2.putText(frame,f"Distance: {dist:.0f} (Click at <{CLICK_DIST})",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),2)

            # Draw click zone around jasmine
            cv2.circle(frame,(int(jasmine_center[0]),int(jasmine_center[1])),CLICK_DIST,(0,255,255),1)

            # Show cooldown status
            time_since_click = time.time() - last_click_time
            if time_since_click < CLICK_COOLDOWN:
                remaining_cooldown = CLICK_COOLDOWN - time_since_click
                cv2.putText(frame,f"Click cooldown: {remaining_cooldown:.1f}s",(10,60),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)

            if auto_enabled:
                # Movement logic
                keys_to_press=[]
                if dx>DEADZONE:
                    keys_to_press.append(RIGHT_KEY)
                elif dx<-DEADZONE:
                    keys_to_press.append(LEFT_KEY)
                if dy>DEADZONE:
                    keys_to_press.append(DOWN_KEY)
                elif dy<-DEADZONE:
                    keys_to_press.append(UP_KEY)
                if keys_to_press:
                    move_keys(keys_to_press,hold=HOLD_TIME)

                # Click logic - when close enough and cooldown expired
                if dist <= CLICK_DIST and time.time() - last_click_time > CLICK_COOLDOWN:
                    success = click_on_jasmine(jasmine_center)
                    if success:
                        # Visual feedback for successful click
                        cv2.circle(frame,(int(jasmine_center[0]),int(jasmine_center[1])),15,(0,255,0),3)
                        cv2.putText(frame,"CLICKED!",(10,90),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,255,0),2)

        # Show status
        status_text = f"Auto: {'ON' if auto_enabled else 'OFF'}"
        cv2.putText(frame,status_text,(10,frame.shape[0]-20),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0) if auto_enabled else (0,0,255),2)

        cv2.imshow("autopilot",frame)
        k=cv2.waitKey(1)&0xFF
        if k==ord('q'):
            break
        elif k==ord('p'):
            auto_enabled=not auto_enabled
            print(f"Auto mode: {'ENABLED' if auto_enabled else 'DISABLED'}")

finally:
    try:
        # Release all keys
        pyautogui.keyUp(UP_KEY)
        pyautogui.keyUp(DOWN_KEY)
        pyautogui.keyUp(LEFT_KEY)
        pyautogui.keyUp(RIGHT_KEY)
    except Exception:
        pass
    cv2.destroyAllWindows()
    print("Auto-pilot stopped")

Starting auto-pilot. Press 'q' to quit, 'p' to toggle auto mode
Current settings: CLICK_DIST=40, CLICK_COOLDOWN=1.0
Clicking jasmine at screen coordinates: (1565, 528)
Auto-pilot stopped


KeyboardInterrupt: 